### 네이버 쇼핑 페이지에서 [전체]-[네이버 랭킹순] 정렬만을 적용한 후 데이터를 수집한다.
---

selenium, webdriver 임포트 및 해당 페이지 주소 가져오기

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('C:/Users/yj/Desktop/datasalon/chromedriver.exe')
url = 'https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&bt=-1&frm=NVSCDIG'
driver.get(url)

In [5]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#### 해당 페이지는 반응형 웹(Responsive Web)으로 구현되어 있어, 스크롤을 내리면 어느 수준까지 창이 늘어나고 정보가 추가된다. 따라서 모든 정보를 노출시킨 다음에 진행했다.

In [6]:
notebooks = soup.select('li.basicList_item__2XT81 > div.basicList_inner__eY_mq > div.basicList_info_area__17Xyo')
len(notebooks)

46

In [7]:
name = soup.select('div.basicList_title__3P9Q7> a')[0].text.strip()
print(name)

삼성전자 갤럭시북 프로 NT950XDY-A51A


In [8]:
price = soup.select('strong.basicList_price__2r23_ > span > span.price_num__2WUXn')[0].text.strip()
## 최저~원으로 뜨는거 정확하게 ~ 부분만 추출
price.split('원')[0].replace(',','')

'1349000'

In [9]:
spec = soup.select('div.basicList_detail_box__3ta3h')[0].text.strip()
spec

'화면크기 : 39.62cm(15.6인치)|무게 : 1.05kg|종류 : 코어i5 11세대|운영체제 : 리눅스|CPU : 코어i5-1135G7|칩셋 제조사 : 인텔|코어종류 : 쿼드코어|코드명 : 타이거레이크|CPU속도 : 2.4GHz|터보부스트 : 4.2GHz|램 : 16GB|메모리 타입 : LPDDR4x|인텔 GPU : Iris Xe Graphics|그래픽 메모리 : 시스템 메모리 공유|해상도 : 1920x1080(FHD)|패널 : AMOLED패널|무선랜 : 802.11 ax(Wi-Fi6)|블루투스 : 블루투스5.1|영상출력 : HDMI, 썬더볼트4|단자 : USB Type C|카드 슬롯 : MicroSD카드|부가기능 : 키보드라이트, 웹캠, 고속충전, MIL-STD|SSD : 256GB|SSD 인터페이스 : NVMe|배터리용량 : 68Wh|사용시간 : 최대20시간|웹카메라 : 전면|스피커 : AKG스테레오스피커|사운드 : 돌비Atoms오디오|품목 : 노트북|두께 : 1.17cm|입력장치 : 숫자키패드|보안기능 : 지문인식, TPM'

In [ ]:
notebook_list = []

for i in notebooks:
    try:
        ## 상품명 가져오기
        name = i.select('div.basicList_title__3P9Q7> a')[0].text.strip()
        ## 가격
        price = i.select('strong.basicList_price__2r23_ > span > span.price_num__2WUXn')[0].text.strip().split('원')[0].replace(',','')
        ## 스펙
        spec = i.select('div.basicList_detail_box__3ta3h')[0].text.strip()
        ## append
        notebook_list.append([name,price,spec])
    except:
        pass

print(len(notebook_list))
notebook_list

In [11]:
## 위 코드 함수

def get_notebooks(ntb):
    notebook_list = []

    for i in notebooks:
        try:
            ## 상품명 가져오기
            name = i.select('div.basicList_title__3P9Q7> a')[0].text.strip()
            ## 가격
            price = i.select('strong.basicList_price__2r23_ > span > span.price_num__2WUXn')[0].text.strip().split('원')[0].replace(',','')
            ## 스펙
            spec = i.select('div.basicList_detail_box__3ta3h')[0].text.strip()
            ## append
            notebook_list.append([name,price,spec])
        except:
            pass
    return notebook_list

In [12]:
# 함수 정상 작동
ntb = soup.select('li.basicList_item__2XT81 > div.basicList_inner__eY_mq > div.basicList_info_area__17Xyo')
len(get_notebooks(ntb))

46

#### 노트북 검색 결과 페이지 url 주소를 살펴본다

1페이지
https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex=1&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list

2페이지
https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex=2&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list

3페이지
https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex=2&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list

다른 검색어 검색(강아지사료)
https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EA%B0%95%EC%95%84%EC%A7%80%EC%82%AC%EB%A3%8C&pagingIndex=1&pagingSize=40&productSet=total&query=%EA%B0%95%EC%95%84%EC%A7%80%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=list

**주소를 비교해보면 origiQuery, pagingIndex, query 부분이 달라지는 것을 볼 수 있다. 다른 검색어를 검색해도 마찬가지이다. `fra=` 프레임 스타일도 다른 검색어 주소와 다르지만 프레임 스타일은 변경해도 주소를 불러오는데 문제가 없었다.**

In [13]:
## 키워드와 페이지 번호를 입력하여 url 주소를 가져오는 함수를 만든다.

def get_search_page_url(keyword,page):
    return 'https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery={}&pagingIndex={}&pagingSize=40&productSet=total&query={}&sort=rel&timestamp=&viewType=list'.format(keyword,page,keyword)

url = get_search_page_url('강아지사료',1)
print(url)


https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery=강아지사료&pagingIndex=1&pagingSize=40&productSet=total&query=강아지사료&sort=rel&timestamp=&viewType=list


In [14]:
import time
from tqdm import tqdm_notebook
total_page = 3
for page in tqdm_notebook(range(1,total_page+1)):
    time.sleep(3)


C:\Users\yj\AppData\Local\Temp/ipykernel_6168/3402917339.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(1,total_page+1)):


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
### 여러 페이지 데이터 수집

keyword = '노트북'
total_page = 3
notebooks_totaldata = []

# 진행 정도를 표현하는 tqdm을 적용
for page in tqdm_notebook(range(1,total_page+1)):
    # 1. 검색 페이지 이동
    url = get_search_page_url('노트북',page)
    driver.get(url)

    # 2. 페이지가 로딩 완료되기 위한 시간으로 10초를 할당
    time.sleep(10)

    # 3. 현재 페이지의 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 4. 상품 정보 추출
    notebooks = soup.select('li.basicList_item__2XT81 > div.basicList_inner__eY_mq > div.basicList_info_area__17Xyo')
    notebook_list = get_notebooks(notebooks)

    ## 5. 추출 데이터 저장
    notebooks_totaldata = notebooks_totaldata + notebook_list

C:\Users\yj\AppData\Local\Temp/ipykernel_6168/2564208047.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(1,total_page+1)):


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
## 엑셀파일로 저장
import pandas as pd
data = pd.DataFrame(notebooks_totaldata)
data.to_excel('C:/Users/yj/Desktop/notebooks.xlsx', index = False)
data.columns = (['상품명','가격','스펙'])

---
---
---

In [ ]:
data = pd.read_excel('C:/Users/yj/Desktop/notebooks.xlsx')
data.columns = (['상품명','가격','스펙'])

In [58]:
## 회사명과 상품명을 분리한다
company_list = []
notebook_list = []
for i in data['상품명']:
    company_list.append(i.split(' ')[0])
    notebook_list.append(i.split(' ',1)[1]) # 첫 번째 공백만 분리

In [59]:
### 테스트
## 무게, 화면크기, 램만 추출해서 비교한다. 노트북을 선택할 때 중요한 요소 중 3가지라고 생각했다.
# 스펙을 종류별로 분리하여 리스트 생성
spec_list = data['스펙'][0].split('|')

for i in spec_list:
    if '무게' in i:
        weight = i
    elif '화면크기 :' in i:
        size = i
    elif '램 ' in i:
        ram = i

print(weight, size, ram)

무게 : 1.05kg 화면크기 : 39.62cm(15.6인치) 램 : 16GB


In [153]:
weight_list = []
size_list = []
ram_list = []

for spec_data in data['스펙']:
    spec_list = spec_data.split('|')
    
    # 정량적 데이터만 추출    
    for i in spec_list:
        if '무게' in i:
            weight = i
            weight = weight.split(':')[1].split('kg')[0].strip()
        if '화면크기 :' in i:
            size = i
            size = size.split(':')[1].split('cm')[0].strip()
        if '램 ' in i:
            ram = i
            ram = ram.split(':')[1].split('G')[0].strip()

    weight_list.append(weight)
    size_list.append(size)
    ram_list.append(ram)

In [154]:
print(weight_list)
print(size_list)
print(ram_list)

['1.05', '1.35', '1.81', '1.35', '1.39', '1.35', '1.05', '1.48', '1.81', '1.39', '1.28', '1.81', '1.15', '1.81', '1.55', '1.19', '1.85', '0.87', '2.2', '1.55', '1.12', '1.65', '1.29', '1.39', '1.3', '1.55', '1.81', '1.04', '1.8', '1.81', '1.3', '1.5', '2.2', '1.53', '1.59', '2.12', '2.5', '1.47', '1.14', '2.4']
['39.62', '43.1', '39.62', '43.1', '35.56', '43.1', '39.62', '40.6', '39.62', '39.62', '33.02', '39.62', '39.62', '39.62', '39.62', '40.6', '39.62', '33.78', '43.94', '39.62', '39.62', '39.62', '33.78', '39.62', '35.81', '39.62', '39.62', '33.78', '39.62', '39.62', '33.78', '39.62', '39.62', '39.62', '39.62', '39.62', '43.94', '35.56', '35.56', '39.62']
['16', '16', '8', '16', '8', '16', '16', '16', '4', '16', '8', '4', '16', '8', '16', '8', '8', '16', '16', '16', '16', '8', '8', '16', '4', '16', '8', '16', '8', '4', '8', '16', '16', '8', '16', '8', '16', '8', '16', '8']


**이제 단위통일을 한다. 무게는 gram으로 계산하고, 화면 크기는 현재 cm인 것을 inch로 변경한다.**

In [155]:
# i.split('.')[0] = 대부분 kg을 뜻함
# i.split('.')[1] = 대부분 gram을 뜻함

for i in weight_list:
    if '.' in i:
        if int(i.split('.')[1]) < 10:
            k = int(i.split('.')[0]) * 1000
            if i.split('.')[1][0] == '0':       # int형으로 비교를 하니 '1.05'같은 변수가 '1.5'로 인식이 된다. 때문에 이러한 조건식을 추가했다.
                g = int(i.split('.')[1]) *10
            else:
                g = int(i.split('.')[1]) * 100
        else:
            k = int(i.split('.')[0]) * 1000
            g = int(i.split('.')[1]) * 10
            
    else:
        k = int(i) * 1000
        g = 0
    print(g+k, end= ' ')

1050 1350 1810 1350 1390 1350 1050 1480 1810 1390 1280 1810 1150 1810 1550 1190 1850 870 2200 1550 1120 1650 1290 1390 1300 1550 1810 1040 1800 1810 1300 1500 2200 1530 1590 2120 2500 1470 1140 2400 

**무게를 gram 단위로 변경했다. 1kg=1000gram로 계산해서 모든 조건을 만족한다.**

In [156]:
# 함수 생성
def trans_weight(weight):
    if '.' in weight:
        if int(weight.split('.')[1]) < 10:
            k = int(weight.split('.')[0]) * 1000
            if weight.split('.')[1][0] == '0':
                g = int(weight.split('.')[1]) * 10
            else:
                g = int(weight.split('.')[1]) * 100
        else:
            g = int(weight.split('.')[1]) * 10
            k = int(weight.split('.')[0]) * 1000
    else:
        k = int(weight) * 1000
        g = 0
    
    return g+k

In [157]:
# 무게 리스트
for i in weight_list:
    print(trans_weight(i), end = ' ')

1050 1350 1810 1350 1390 1350 1050 1480 1810 1390 1280 1810 1150 1810 1550 1190 1850 870 2200 1550 1120 1650 1290 1390 1300 1550 1810 1040 1800 1810 1300 1500 2200 1530 1590 2120 2500 1470 1140 2400 

In [182]:
# 변환한 무게 데이터 값을 새로운 리스트에 할당한다.
new_weight_list =  []
for i in weight_list:
    value1 = trans_weight(i)
    new_weight_list.append(value1)

print(new_weight_list, end=' ')

[1050, 1350, 1810, 1350, 1390, 1350, 1050, 1480, 1810, 1390, 1280, 1810, 1150, 1810, 1550, 1190, 1850, 870, 2200, 1550, 1120, 1650, 1290, 1390, 1300, 1550, 1810, 1040, 1800, 1810, 1300, 1500, 2200, 1530, 1590, 2120, 2500, 1470, 1140, 2400] 

**마찬가지로 cm도 inch로 변경한다. 1 cm = 0.393701 inch.**

In [178]:
for i in size_list:
    if '.' in size:
        cm = int(i.split('.')[0])
        mm = int(i.split('.')[1]) / 100
    else:
        cm = int(i)
        mm = 0

    print(round((cm+mm)*0.393701,1), end = ' ')

15.6 16.9 15.6 16.9 14.0 16.9 15.6 15.8 15.6 15.6 13.0 15.6 15.6 15.6 15.6 15.8 15.6 13.3 17.3 15.6 15.6 15.6 13.3 15.6 14.1 15.6 15.6 13.3 15.6 15.6 13.3 15.6 15.6 15.6 15.6 15.6 17.3 14.0 14.0 15.6 

In [179]:
# 함수 생성

def trans_size(size):
    if '.' in size:
        cm = int(size.split('.')[0])
        mm = int(size.split('.')[1]) / 100
    else:
        cm = int(size)
        mm = 0
    
    return round((cm+mm)*0.393701,1)

In [183]:
# 새로운 리스트에 할당
new_size_list = []
for i in size_list:
    value2 = trans_size(i)
    new_size_list.append(value2)

print(new_size_list, end=' ')

[15.6, 16.9, 15.6, 16.9, 14.0, 16.9, 15.6, 15.8, 15.6, 15.6, 13.0, 15.6, 15.6, 15.6, 15.6, 15.8, 15.6, 13.3, 17.3, 15.6, 15.6, 15.6, 13.3, 15.6, 14.1, 15.6, 15.6, 13.3, 15.6, 15.6, 13.3, 15.6, 15.6, 15.6, 15.6, 15.6, 17.3, 14.0, 14.0, 15.6] 

**ram_list의 값도 정수형으로 변경하자**

In [135]:
print(ram_list, end ='')

['16', '16', '8', '16', '8', '16', '16', '16', '4', '16', '8', '4', '16', '8', '16', '8', '8', '16', '16', '16', '16', '8', '8', '16', '4', '16', '8', '16', '8', '4', '8', '16', '16', '8', '16', '8', '16', '8', '16', '8']

In [184]:
def trans_ram(ram):
    return int(ram)

In [185]:
new_ram_list = []
for i in ram_list:
    value3 = trans_ram(i)
    new_ram_list.append(value3)

print(new_ram_list, end = ' ')

[16, 16, 8, 16, 8, 16, 16, 16, 4, 16, 8, 4, 16, 8, 16, 8, 8, 16, 16, 16, 16, 8, 8, 16, 4, 16, 8, 16, 8, 4, 8, 16, 16, 8, 16, 8, 16, 8, 16, 8] 

**전처리 결과를 엑셀로 저장**

In [189]:
pd_data = pd.DataFrame()
pd_data['회사명'] = company_list
pd_data['상품명'] = notebook_list
pd_data['가격'] = data['가격']
pd_data['무게'] = new_weight_list
pd_data['화면크기'] = new_size_list
pd_data['램'] = new_ram_list

pd_data.to_excel('C:/Users/yj/Desktop/naver_shopping_notebook.xlsx',index = False)